# Imports and setup

In [ ]:
!pip install copulae
!pip install skfeature-chappers
!pip install scikit-posthocs

from copulae import EmpiricalCopula, pseudo_obs
import numpy as np
import pandas as pd
import csv
from math import fabs
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, matthews_corrcoef, roc_auc_score, auc, average_precision_score, precision_recall_curve
from sklearn.model_selection import LeavePOut, LeaveOneOut, KFold, StratifiedKFold, GridSearchCV
from sklearn.feature_selection import f_classif, SelectFdr, RFE, VarianceThreshold, SelectKBest
from sklearn.svm import SVR, SVC
from skfeature.function.information_theoretical_based.MRMR import mrmr
from skfeature.function.similarity_based.reliefF import reliefF
from timeit import default_timer as time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PowerTransformer
from os.path import exists, isdir
from sklearn.base import BaseEstimator
from sklearn.impute import SimpleImputer
from sklearn.decomposition import KernelPCA
from google.colab import drive
from imblearn.over_sampling import BorderlineSMOTE
from scipy.stats import friedmanchisquare, wilcoxon
from scikit_posthocs import posthoc_nemenyi_friedman as posthoc
from collections import Counter
from os import listdir, remove, chdir

In [ ]:
drive.mount('/content/drive')
chdir("./drive/MyDrive/Hishuvit4/")

# Part A

In [3]:
class CBFS:
  def __init__(self, k):
    # Only one hyper parameter - k - the number of features we want to choose
    self.k = k

  def Ic(self, ec, u):
    # The Ic function
    res = np.mean(ec.cdf(u))
    return np.abs(np.log2(res) * res if res != 0. else 0.)

  def fit(self, X, y):
    # The algorithm described in the paper "Stable feature selection using copula based mutual information"
    n, col = X.shape
    u = np.random.uniform(size=(n, 2))
    temp = np.c_[y, X]
    mimc1 = np.array([self.Ic(EmpiricalCopula(pseudo_obs(temp[:, [0, j + 1]])), u) for j in range(col)]) # Mutual information of all features with y
    output = [np.argmax(mimc1)]
    for m in range(1, self.k):
      u = np.random.uniform(size=(n, m + 1))
      temp = np.c_[X.iloc[:, output] if type(X) == pd.DataFrame else X[:, output], X]
      red = np.array([self.Ic(EmpiricalCopula(pseudo_obs(temp[:, list(range(m)) + [m + j]])), u) if j not in output else np.NINF for j in range(col)]) # The mutual information of each feature with the currently selected features
      result = mimc1 - red # Subtract the mutual information of each feature with the currently selected features from the mutual information of each feature with y
      output.append(np.argmin(result)) # Add the best result to the output
    self.mask_ = np.zeros(col, dtype=np.int32)
    self.mask_[output] = 1
    self.scores_ = self.mask_
    return self

  def transform(self, X):
    if (self.mask_ is None):
        raise Exception("Can't transform data without fitting first")
    if (type(X) == pd.DataFrame):
      return X[X.columns[self.mask_ == 1]]
    return X[self.mask_ == 1]

  def fit_transform(self, X, y):
    self.fit(X, y)
    return self.transform(X)

In [4]:
class XSpace:
  def __init__(self, X, c, k):
    self.X = X
    self.c = c
    self.k = k
    self.cache_neighbors = dict()
  
  def is_nearest_neighbors(self, i, j, cFlag=None):
    if (i in self.cache_neighbors):
      nearest_neighbors_ids = self.cache_neighbors[i]
    else:
      X_i = np.delete(self.X, i, axis=0)
      nearest_neighbors_ids = np.linalg.norm(self.X[i] - X_i, axis=1).argsort()[:self.k]
      self.cache_neighbors[i] = nearest_neighbors_ids
    if(cFlag == None):
      return j in nearest_neighbors_ids
    elif(cFlag == "w"):
      return j in nearest_neighbors_ids and self.c[i] == self.c[j]
    else:
      return j in nearest_neighbors_ids and self.c[i] != self.c[j]


class LSLSFS:
  def __init__(self, k):
    # Only one hyper parameter - k - the number of features we want to choose
    # Note: if we had enough computation power, alpha could've been a hyper parameter as well, but sadly we do not have access to good computation power
    self.k = k
    self.alpha = 0.5
    
  def compute_W(self, xSpace, X, m, cFlag=None):
    return np.array([[np.abs(X[i].T @ X[j]) / (np.linalg.norm(X[i]) * np.linalg.norm(X[j])) if xSpace.is_nearest_neighbors(i, j, cFlag) or xSpace.is_nearest_neighbors(j, i, cFlag) else 0 for j in range(m)] for i in range(m)])

  def compute_D(self, W, m):
    return np.diag(np.array([sum(W[:, i]) for i in range(m)]))

  def compute_L(self, W, D):
    return D - W

  def compute_fr(self, X, D, r, m):
    fr = X[:, r]
    ones = np.ones(m)
    numerator = fr @ D @ ones
    denominator = ones @ D @ ones
    return fr.T - ((numerator/denominator) * ones)

  def compute_LSLSr(self, fr, Lw, Lb, D):
    aLw = self.alpha * Lw
    aLb = (1 - self.alpha) * Lb
    numerator = fr.T @ (aLw - aLb) @ fr
    denominator = fr.T @ D @ fr
    return numerator / denominator

  def fit(self, X, y=None):
    XT = X.to_numpy() if type(X) == pd.DataFrame else X
    m, n = X.shape
    xSpace = XSpace(XT, y, self.k)
    W = self.compute_W(xSpace, XT, m)
    D = self.compute_D(W, m)

    Ww = self.compute_W(xSpace, XT, m, cFlag="w")
    Dw = self.compute_D(Ww, m)
    Lw = self.compute_L(Ww, Dw)

    Wb = self.compute_W(xSpace, XT, m, cFlag="b")
    Db = self.compute_D(Wb, m)
    Lb = self.compute_L(Wb, Db)

    scores = np.array([self.compute_LSLSr(self.compute_fr(XT, D, r, m), Lw, Lb, D) for r in range(n)])

    self.scores_ = scores
    self.mask_ = np.zeros(n, dtype=np.int32)
    self.mask_[np.argsort(self.scores_)[::-1][:self.k]] = 1
    return self
    
  def transform(self, X):
    if (self.mask_ is None):
        raise Exception("Can't transform data without fitting first")
    if (type(X) == pd.DataFrame):
      return X[X.columns[self.mask_ == 1]]
    return X[self.mask_ == 1]

  def fit_transform(self, X, y):
    self.fit(X, y)
    return self.transform(X)

In [ ]:
train_df = pd.read_csv(f"Data/SPECTF.train", header=None)
X_train = train_df.iloc[:, 1:]
y_train = train_df.iloc[:, 0]

test_df = pd.read_csv(f"Data/SPECTF.test", header=None)
X_test = test_df.iloc[:, 1:]
y_test = test_df.iloc[:, 0]

base_model = RandomForestClassifier()
base_model.fit(X_train, y_train)
print(f"Model accuracy without feature selection: {accuracy_score(base_model.predict(X_test), y_test)}")
for k in range(5, 15, 3):
  model = make_pipeline(CBFS(k), RandomForestClassifier())
  model.fit(X_train, y_train)
  print(f"Model accuracy with CBFS with k={k}: {accuracy_score(model.predict(X_test), y_test)}")

for k in range(5, 15, 3):
  model = make_pipeline(LSLSFS(k), RandomForestClassifier())
  model.fit(X_train, y_train)
  print(f"Model accuracy with LSLSFS with k={k}: {accuracy_score(model.predict(X_test), y_test)}")

# Part C

In [5]:
class New_CBFS:
  def __init__(self, k):
    # Only one hyper parameter - k - the number of features we want to choose
    # Note: if we had enough computation power, q could've been a hyper parameter as well, but sadly we do not have access to good computation power
    self.k = k
    self.q = 0.3

  def Ic(self, ec, u):
    # Tsallis Entropy based Ic calculation
    return np.mean(1 - (ec.cdf(u) ** self.q)) / (self.q - 1)

  def fit(self, X, y):
    n, col = X.shape
    u = np.random.uniform(size=(n, 2))
    temp = np.c_[y, X]
    mimc1 = np.array([self.Ic(EmpiricalCopula(pseudo_obs(temp[:, [0, j + 1]])), u) for j in range(col)]) # Mutual information of all features with y
    output = [np.argmax(mimc1)]
    for m in range(1, self.k):
      u = np.random.uniform(size=(n, m + 1))
      temp = np.c_[X.iloc[:, output] if type(X) == pd.DataFrame else X[:, output], X]
      red = np.array([self.Ic(EmpiricalCopula(pseudo_obs(temp[:, list(range(m)) + [m + j]])), u) if j not in output else np.NINF for j in range(col)]) # The mutual information of each feature with the currently selected features
      result = mimc1 - red # Subtract the mutual information of each feature with the currently selected features from the mutual information of each feature with y
      output.append(np.argmin(result)) # Add the best result to the output
    self.mask_ = np.zeros(col, dtype=np.int32)
    self.mask_[output] = 1
    self.scores_ = self.mask_
    return self

  def transform(self, X):
    if (self.mask_ is None):
        raise Exception("Can't transform data without fitting first")
    if (type(X) == pd.DataFrame):
      return X[X.columns[self.mask_ == 1]]
    return X[self.mask_ == 1]

  def fit_transform(self, X, y):
    self.fit(X, y)
    return self.transform(X)

In [ ]:
train_df = pd.read_csv(f"Data/SPECTF.train", header=None)
X_train = train_df.iloc[:, 1:]
y_train = train_df.iloc[:, 0]

test_df = pd.read_csv(f"Data/SPECTF.test", header=None)
X_test = test_df.iloc[:, 1:]
y_test = test_df.iloc[:, 0]

base_model = RandomForestClassifier()
base_model.fit(X_train, y_train)
print(f"Model accuracy without feature selection: {accuracy_score(base_model.predict(X_test), y_test)}")
for k in range(5, 15, 3):
  model = make_pipeline(New_CBFS(k), RandomForestClassifier())
  model.fit(X_train, y_train)
  print(f"Model accuracy with New_CBFS with k={k}: {accuracy_score(model.predict(X_test), y_test)}")

# Part B and C experiments

In [6]:
class ReliefF:
  def __init__(self, k):
    self.k = k
    
  def fit(self, X, y):
    n, col = X.shape
    self.n_features_in_ = col
    if type(X) == pd.DataFrame:
      X = X.to_numpy()
    if type(y) == pd.DataFrame:
      y = y.to_numpy()
    self.scores_ = reliefF(X, y, mode="raw", k=self.k)
    self.mask_ = np.zeros(col, dtype=np.int32)
    self.mask_[np.argsort(self.scores_, 0)[::-1][:self.k]] = 1
    return self

  def transform(self, X, y=None):
    if (self.mask_ is None):
      raise Exception("Can't transform data without fitting first")
    if (type(X) == pd.DataFrame):
      return X[X.columns[self.mask_ == 1]]
    return X[self.mask_ == 1]

  def fit_transform(self, X, y):
    self.fit(X, y)
    return self.transform(X)


class MRMR:
  def __init__(self, k):
    self.k = k

  def fit(self, X, y):
    n, col = X.shape
    self.n_features_in_ = col
    if type(X) == pd.DataFrame:
      X = X.to_numpy()
    if type(y) == pd.DataFrame:
      y = y.to_numpy()
    self.fs = mrmr(X, y, mode='index', n_selected_features=self.k)
    self.mask_ = np.zeros(col, dtype=np.int32)
    self.mask_[self.fs] = 1
    self.scores_ = self.mask_
    return self

  def transform(self, X, y=None):
    if (self.mask_ is None):
      raise Exception("Can't transform data without fitting first")
    if (type(X) == pd.DataFrame):
      return X[X.columns[self.mask_ == 1]]
    return X[self.mask_ == 1]

  def fit_transform(self, X, y):
    self.fit(X, y)
    return self.transform(X)


class CustomFDR:
  def __init__(self, k):
    pass

  def fit(self, X, y):
    n, col = X.shape
    self.n_features_in_ = col
    if type(X) == pd.DataFrame:
      X = X.to_numpy()
    if type(y) == pd.DataFrame:
      y = y.to_numpy()
    self.fs = SelectFdr(score_func=f_classif, alpha=0.1)
    self.fs.fit(X, y)
    self.scores_ = self.fs.scores_
    self.mask_ = np.zeros(col, dtype=np.int32)
    temp = self.fs.get_support()
    if True in temp:
      self.mask_[temp] = 1
    else:
      self.mask_[np.argmax(self.scores_)] = 1
    return self

  def transform(self, X, y=None):
    if (self.mask_ is None):
      raise Exception("Can't transform data without fitting first")
    if (type(X) == pd.DataFrame):
      return X[X.columns[self.mask_ == 1]]
    return X[self.mask_ == 1]

  def fit_transform(self, X, y):
    self.fit(X, y)
    return self.transform(X)

In [7]:
MEASURE_TYPES = ["ACC", "PR-AUC", "AUC", "MCC"]

header = ['Dataset Name', 'Number of samples', 'Original number of features', 
                  'Filtering Algorithm', 'Learning Algorithm', 'Number of features selected (K)', 
                  'CV Method', 'Fold', 'Measure Type', 'Measure Value', 'List of Selected Features Names', 
                  'Selected Features Scores', 'Training time (whole pipeline)', 'Testing time']

In [8]:
TRUE_CLASS_LABEL_INDEX = 1

def multiclass_to_binary(y, c):
  return (y == c).astype(int)

def write_to_results(RESULTS_FILENAME, DATASET_NAME, NUMBER_OF_SAMPLES,
                     ORIGINAL_NUMBER_OF_FEATURES, FILTERING_ALGORITHM, 
                     LEARNING_ALGORITHM, NUMBER_OF_FEATURES_SELECTED, 
                     CV_METHOD, FOLD, 
                     MEASURE_VALUES, SELECTED_FEATURES_NAMES, 
                     SELECTED_FEATURES_SCORES, PIPELINE_TIME, TEST_TIME):
  if not exists(RESULTS_FILENAME):
    with open(RESULTS_FILENAME, 'w+', encoding='UTF8') as f:
      writer = csv.writer(f)
      writer.writerow(header)
  
  rows = [[DATASET_NAME, NUMBER_OF_SAMPLES, ORIGINAL_NUMBER_OF_FEATURES, 
         FILTERING_ALGORITHM, LEARNING_ALGORITHM, NUMBER_OF_FEATURES_SELECTED,
         CV_METHOD, FOLD, MEASURE_TYPE, MEASURE_VALUE, SELECTED_FEATURES_NAMES,
         SELECTED_FEATURES_SCORES, PIPELINE_TIME, TEST_TIME] for MEASURE_TYPE, MEASURE_VALUE in zip(MEASURE_TYPES, MEASURE_VALUES)]
  with open(RESULTS_FILENAME, 'a', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(rows)

class Switch:
  def __init__(self, datasetname=None, preprocessing=None, fs=None, estimator=None, k=None, results_file=None):
      self.preprocessing = preprocessing
      self.fs = fs
      self.estimator = estimator
      self.datasetname = datasetname
      self.k = k
      self.results_file = results_file

  def get_params(self, deep=True):
      return {'preprocessing': self.preprocessing,
              'fs': self.fs,
              'estimator': self.estimator,
              'datasetname': self.datasetname,
              'k': self.k,
              'results_file': self.results_file}

  def set_params(self, **params):
    if not params:
      return self

    for key, value in params.items():
      if hasattr(self, key):
        setattr(self, key, value)
      else:
        self.kwargs[key] = value
    return self

  def __getstate__(self):
      state = self.__dict__.copy()
      return state

  def __setstate__(self, state):
      self.__dict__.update(state)

  def get_features(self, X_train):
    if hasattr(self.fs, 'support_'):
      feature_names = ", ".join(str(n) for n in (X_train[X_train.columns[self.fs.support_ == 1]] if type(X_train) == pd.DataFrame else X_train[self.fs.support_ == 1]))
      if hasattr(self.fs, 'scores_'):
        feature_scores = ", ".join(str(n) for n in self.fs.scores_[self.fs.support_ == 1])
      elif hasattr(self.fs, 'ranking_'):
        feature_scores = ", ".join(str(n) for n in self.fs.ranking_[self.fs.support_ == 1])
      else:
        raise Exception(f"No scores_ and no ranking_: {self.FILTERING_ALGORITHM}")

    elif hasattr(self.fs, 'mask_'):
      feature_names = ", ".join(str(n) for n in (X_train[X_train.columns[self.fs.mask_ == 1]] if type(X_train) == pd.DataFrame else X_train[self.fs.mask_ == 1]))
      if hasattr(self.fs, 'scores_'):
        feature_scores = ", ".join(str(n) for n in self.fs.scores_[self.fs.mask_ == 1])
      elif hasattr(self.fs, 'ranking_'):
        feature_scores = ", ".join(str(n) for n in self.fs.ranking_[self.fs.mask_ == 1])
      else:
        raise Exception(f"No scores_ and no ranking_: {self.FILTERING_ALGORITHM}")

    else:
      raise Exception(f"No mask_ and no support_: {self.FILTERTING_ALGORITHM}")
    return feature_names, feature_scores

  def evaluate(self, X_train, X_test, y_train, y_test):
    _, y_train = np.unique(y_train, return_inverse=True)
    _, y_test = np.unique(y_test, return_inverse=True)
    PIPELINE_TIME = time()
    self.current_model.fit(X_train, y_train)
    PIPELINE_TIME = time() - PIPELINE_TIME
    
    testing_start_time = time()
    y_pred = self.current_model.predict(X_test)
    y_pred_proba = self.current_model.predict_proba(X_test)

    acc = accuracy_score(y_test, y_pred)

    temp = np.unique(y_test)

    if len(temp) == 1: # Only one class in the test set
      rocauc = acc
      prauc = average_precision_score(y_test, y_pred_proba[:, 0])
    elif len(temp) == 2: # Binary classification in the test set
      rocauc = roc_auc_score(y_test, y_pred_proba[:, TRUE_CLASS_LABEL_INDEX])
      prauc = average_precision_score(y_test, y_pred_proba[:, TRUE_CLASS_LABEL_INDEX])
    else: # Multi-class
      rocauc = roc_auc_score(y_test, y_pred_proba, multi_class='ovo', labels=list(set(np.unique(y_train)).union(set(np.unique(y_test)))))
      prauc = sum(average_precision_score(multiclass_to_binary(y_test, c), y_pred_proba[:, c]) for c in temp) / len(temp)
    mcc = matthews_corrcoef(y_test, y_pred)
    TESTING_TIME = time() - testing_start_time
    feature_names, feature_scores = self.get_features(X_train)
    return str(PIPELINE_TIME), str(TESTING_TIME), feature_names, feature_scores, [str(acc), str(prauc), str(rocauc), str(mcc)]

  def fit(self, X, y=None):
    NUMBER_OF_SAMPLES, ORIGINAL_NUMBER_OF_FEATURES = X.shape
    FILTERING_ALGORITHM = self.fs[0]
    LEARNING_ALGORITHM = self.estimator[0]
    self.fs = self.fs[1](self.k)
    self.estimator = self.estimator[1]()
    DATASET_NAME = self.datasetname
    NUMBER_OF_FEATURES_SELECTED = self.k
    self.current_model = make_pipeline(self.preprocessing, self.fs, self.estimator)
    if NUMBER_OF_SAMPLES < 50:
      cv, CV_METHOD = LeavePOut(2), "Leave-pair-out"
    elif 50 < NUMBER_OF_SAMPLES < 100:
      cv, CV_METHOD = LeaveOneOut(), "Leave-one-out"
    elif 100 < NUMBER_OF_SAMPLES < 1000:
      cv, CV_METHOD = StratifiedKFold(n_splits=10), "10-Folds cv"
    else:
      cv, CV_METHOD = StratifiedKFold(n_splits=5), "5-Folds cv"

    for fold_num, (train_index, test_index) in enumerate(cv.split(X, y)):
      if type(X) == pd.DataFrame:
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
      else:
        X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]

      PIPELINE_TIME, MODEL_TRAINING_TIME, TEST_TIME, SELECTED_FEATURES_NAMES, SELECTED_FEATURES_SCORES, MEASURE_VALUES = self.evaluate(X_train, X_test, y_train, y_test)

      FOLD = str(fold_num + 1)
      write_to_results(self.results_file, DATASET_NAME, NUMBER_OF_SAMPLES, ORIGINAL_NUMBER_OF_FEATURES, FILTERING_ALGORITHM, LEARNING_ALGORITHM, NUMBER_OF_FEATURES_SELECTED, CV_METHOD, FOLD, MEASURE_VALUES, SELECTED_FEATURES_NAMES, SELECTED_FEATURES_SCORES, PIPELINE_TIME, TEST_TIME)
    return self

  def predict(self, X, y=None):
    return self.current_model.predict(X)

  def predict_proba(self, X):
    return self.current_model.predict_proba(X)

  def score(self, X, y):
    return self.current_model.score(X, y)


In [9]:
FS_NAMES = ['RFE_USING_SVM',
            'F_CLASSIF_FDR10%',
            'MRMR',
            'ReliefF',
            'New_CBFS',
            'CBFS',
            'LSLSFS']

FS_METHODS = [lambda k: RFE(SVR(kernel="linear"), n_features_to_select=k),
            CustomFDR,
            MRMR,
            ReliefF,
            New_CBFS,
            CBFS,
            LSLSFS]

K_VALUES = [1, 2, 3, 4, 5, 10, 15, 20, 25, 30, 50, 100]

CLFS_NAMES = ['KNN',
              'Gaussian Naive Bayes',
              'SVM',
              'Logistic Regression',
              'Random Forest']

CLFS = [KNeighborsClassifier,
        GaussianNB,
        lambda: SVC(kernel='linear', gamma='auto', probability=True),
        LogisticRegression,
        RandomForestClassifier]

In [10]:
def full_comparison_part_b(X, y, dataset_name, results_filename, preprocess_filename, preprocessing=None):
  curr_preprocessing = make_pipeline(SimpleImputer(), VarianceThreshold(), PowerTransformer())
  REDUCE_FEATURES = X.shape[1] > 1000
  if REDUCE_FEATURES:
    curr_preprocessing.steps.append(['initial_feature_reducer', SelectKBest(k=1000)])
  if preprocessing is not None:
    curr_preprocessing.steps.append(['custom_preprocessing', preprocessing])
  X_new = curr_preprocessing.fit_transform(X, y)
  new_cols = X.columns[curr_preprocessing['initial_feature_reducer'].get_support()] if REDUCE_FEATURES else X.columns
  X = pd.DataFrame(X_new, columns=new_cols, index=X.index)
  X.to_csv(preprocess_filename)
  parameters = {'datasetname': [dataset_name],
                'fs': list(zip(FS_NAMES, FS_METHODS)),
                'estimator': list(zip(CLFS_NAMES, CLFS)),
                'k': K_VALUES,
                'results_file': [results_filename]}

  gscv = GridSearchCV(Switch(), parameters, n_jobs=-1)
  gscv.fit(X, y)

def get_x_y(ds_group, ds_name):
  if ds_group == 'mAMLData':
    df = pd.read_csv(f"Data/{ds_group}/{ds_name}/{ds_name}.csv", index_col = 0)
    X = df
    samples_names = X.index
    df_labels = pd.read_csv(f"Data/{ds_group}/{ds_name}/{ds_name}.mf.csv")
    y = np.array([df_labels.loc[df_labels['#SampleID'] == samples_names[index]]["label"].iloc[0] for index in range(X.shape[0])])
  elif ds_group == 'bioconductor':
    df = pd.read_csv(f"Data/{ds_group}/{ds_name}.csv", index_col=0).T
    X = df.iloc[:, 1:]
    y = df.iloc[:, 0].to_numpy(dtype=np.int32)
  elif ds_group == 'datamicroarray':
    X = pd.read_csv(f"Data/{ds_group}/{ds_name}_inputs.csv")
    y = pd.read_csv(f"Data/{ds_group}/{ds_name}_outputs.csv").to_numpy(dtype=np.int32).ravel()
  elif ds_group == 'microbiomicData':
    df = pd.read_csv(f"Data/{ds_group}/{ds_name}.csv")
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1].values
  return X, y

def run_part_b(ds_group, ds_name):
  X, y = get_x_y(ds_group, ds_name)
  results_filename = f"Results/{ds_group}/{ds_name}/{ds_name}_res.csv"
  preprocess_filename = f"Results/{ds_group}/{ds_name}/{ds_name}_PROCESSED.csv"
  if not exists(results_filename):
    with open(results_filename, 'w+', encoding='UTF8') as f:
      writer = csv.writer(f)
      writer.writerow(header)

  full_comparison_part_b(X, y, ds_name, results_filename, preprocess_filename)

In [ ]:
for ds_group in listdir('Data'):
  if isdir(f"Data/{ds_group}"): # We don't want SPECTF.test, SPECTF.train
    for ds_name in listdir(f"Data/{ds_group}"):
      run_part_b(ds_group, ds_name[:-4])

# Part D

In [11]:
header_part_d = ['Dataset Name', 'Number of samples', 'Original number of features', 
                  'Filtering Algorithm', 'Learning Algorithm', 'Number of features selected (K)', 
                  'CV Method', 'Fold', 'Measure Type', 'Measure Value', 'Training time (whole pipeline)', 'Testing time']

FS = dict(zip(FS_NAMES, FS_METHODS))

CLFS_METHODS = dict(zip(CLFS_NAMES, CLFS))

def write_to_results_part_d(RESULTS_FILENAME, DATASET_NAME, NUMBER_OF_SAMPLES,
                     ORIGINAL_NUMBER_OF_FEATURES, FILTERING_ALGORITHM, 
                     LEARNING_ALGORITHM, NUMBER_OF_FEATURES_SELECTED, 
                     CV_METHOD, FOLD,
                     MEASURE_VALUES, PIPELINE_TIME, TEST_TIME):
  if not exists(RESULTS_FILENAME):
    with open(RESULTS_FILENAME, 'w+', encoding='UTF8') as f:
      writer = csv.writer(f)
      writer.writerow(header_part_d)

  rows = [[DATASET_NAME, NUMBER_OF_SAMPLES, ORIGINAL_NUMBER_OF_FEATURES, 
         FILTERING_ALGORITHM, LEARNING_ALGORITHM, NUMBER_OF_FEATURES_SELECTED,
         CV_METHOD, FOLD, MEASURE_TYPE, MEASURE_VALUE, PIPELINE_TIME, TEST_TIME] for MEASURE_TYPE, MEASURE_VALUE in zip(MEASURE_TYPES, MEASURE_VALUES)]
  with open(RESULTS_FILENAME, 'a', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(rows)

def run_part_d(X, y, dataset_name, res_file_name):
    RESULTS_FILENAME = f"Results_PartD/{dataset_name}_res_aug.csv"
    print(f"Reading file {res_file_name}...")
    res_df = pd.read_csv(res_file_name, header=0)
    best_auc_row_idx = res_df[res_df["Measure Type"] == 'AUC']["Measure Value"].idxmax()
    best_auc_row = res_df.iloc[best_auc_row_idx]
    winning_fs = best_auc_row["Filtering Algorithm"]
    winning_k = best_auc_row["Number of features selected (K)"]
    winning_clf = best_auc_row["Learning Algorithm"]
    fs_algo = FS[winning_fs](winning_k)
    # Part D - A starts here!
    with open("PartD_ChosenConfigurations.txt", "a+") as f:
      f.write(f"For dataset {dataset_name}, the best configuration is feature selection algorithm {winning_fs}, number of features (k) {winning_k} and classifier {winning_clf}\n")
      print(f"For dataset {dataset_name}, the best configuration is feature selection algorithm {winning_fs}, number of features (k) {winning_k} and classifier {winning_clf}")

    X_new = fs_algo.fit_transform(X, y)
    X = pd.DataFrame(X_new, index=X.index)
    part_d(X, y, winning_fs, fs_algo, winning_clf, dataset_name, winning_k, RESULTS_FILENAME)
   
    
def evaluate_part_d(X_train, X_test, y_train, y_test, model):
    _, y_train = np.unique(y_train, return_inverse=True)
    _, y_test = np.unique(y_test, return_inverse=True)

    TRAINING_TIME = time()
    model.fit(X_train, y_train)
    TRAINING_TIME = time() - TRAINING_TIME
    
    TESTING_TIME = time()
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)

    acc = accuracy_score(y_test, y_pred)

    temp = np.unique(y_test)

    if len(temp) == 1: # Only one class in the test set
      rocauc = acc
      prauc = average_precision_score(y_test, y_pred_proba[:, 0])
    elif len(temp) == 2: # Binary classification in the test set
      rocauc = roc_auc_score(y_test, y_pred_proba[:, TRUE_CLASS_LABEL_INDEX])
      prauc = average_precision_score(y_test, y_pred_proba[:, TRUE_CLASS_LABEL_INDEX])
    else: # Multi-class
      rocauc = roc_auc_score(y_test, y_pred_proba, multi_class='ovo', labels=list(set(np.unique(y_train)).union(set(np.unique(y_test)))))
      prauc = sum(average_precision_score(multiclass_to_binary(y_test, c), y_pred_proba[:, c]) for c in temp) / len(temp)
    mcc = matthews_corrcoef(y_test, y_pred)
    TESTING_TIME = time() - TESTING_TIME
    
    return str(TRAINING_TIME), str(TESTING_TIME), [str(acc), str(prauc), str(rocauc), str(mcc)]
    
def part_d(X, y, FILTERING_ALGORITHM, fs, LEARNING_ALGORITHM, DATASET_NAME, k, results_file):
    NUMBER_OF_SAMPLES, ORIGINAL_NUMBER_OF_FEATURES = X.shape
    NUMBER_OF_FEATURES_SELECTED = k
    if NUMBER_OF_SAMPLES < 50:
      cv, CV_METHOD = LeavePOut(2), "Leave-pair-out"
    elif 50 < NUMBER_OF_SAMPLES < 100:
      cv, CV_METHOD = LeaveOneOut(), "Leave-one-out"
    elif 100 < NUMBER_OF_SAMPLES < 1000:
      cv, CV_METHOD = StratifiedKFold(n_splits=10), "10-Folds cv"
    else:
      cv, CV_METHOD = StratifiedKFold(n_splits=5), "5-Folds cv"

    for fold_num, (train_index, test_index) in enumerate(cv.split(X, y)):
      current_model = CLFS_METHODS[LEARNING_ALGORITHM]()
      if type(X) == pd.DataFrame:
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
      else:
        X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]

      # Part D - B starts here!
      transformer_linear = KernelPCA(kernel='linear')
      transformer_rbf = KernelPCA(kernel='rbf')
      X_train_transformed_linear = transformer_linear.fit_transform(X_train)
      X_train_transformed_rbf = transformer_rbf.fit_transform(X_train)
      X_train = np.concatenate((X_train, X_train_transformed_linear, X_train_transformed_rbf), axis=1)

      # Part D - C starts here!
      X_test_transformed_linear = transformer_linear.transform(X_test)
      X_test_transformed_rbf = transformer_rbf.transform(X_test)
      X_test = np.concatenate((X_test, X_test_transformed_linear, X_test_transformed_rbf), axis=1)

      # Part D - D starts here!
      neighbors_limit = min(Counter(y_train).values()) - 1
      X_train, y_train = BorderlineSMOTE(k_neighbors=min(neighbors_limit, 5), m_neighbors=min(neighbors_limit, 10), n_jobs=-1).fit_resample(X_train, y_train)

      # Part D - E starts here!
      PIPELINE_TIME, TEST_TIME, MEASURE_VALUES = evaluate_part_d(X_train, X_test, y_train, y_test, current_model)

      # Part D - F starts here!
      FOLD = str(fold_num + 1)
      write_to_results_part_d(results_file, DATASET_NAME, NUMBER_OF_SAMPLES, ORIGINAL_NUMBER_OF_FEATURES, FILTERING_ALGORITHM, LEARNING_ALGORITHM, NUMBER_OF_FEATURES_SELECTED, CV_METHOD, FOLD, MEASURE_VALUES, PIPELINE_TIME, TEST_TIME)


In [ ]:
for ds_group in listdir('Results'):
  for ds_name in listdir(f"Results/{ds_group}"):
    if ds_name not in ["ayeastCC", "west", "singh", "CSS", "PBS", "BP", "PDX"]: # Last 4 crash because not enough RAM, first 3 are too large to read as pandas dataframe! :(
      results_file_name = f"Results/{ds_group}/{ds_name}/{ds_name}_res.csv"
      X, y = get_x_y(ds_group, ds_name)
      # Run the same preprocessing as we run in Part B
      curr_preprocessing = make_pipeline(SimpleImputer(), VarianceThreshold(), PowerTransformer())
      REDUCE_FEATURES = X.shape[1] > 1000
      if REDUCE_FEATURES:
        curr_preprocessing.steps.append(['initial_feature_reducer', SelectKBest(k=1000)])
      X_new = curr_preprocessing.fit_transform(X, y)
      new_cols = X.columns[curr_preprocessing['initial_feature_reducer'].get_support()] if REDUCE_FEATURES else X.columns
      X = pd.DataFrame(X_new, columns=new_cols, index=X.index)

      run_part_d(X, y, ds_name, results_file_name)

We want to analyze the results - we want to compare the results of part d to the actual results. The code block below handles that!

In [ ]:
part_b_won = []
part_d_won = []
maximum_improved = []

for ds_group in listdir('Results'):
  for ds_name in listdir(f"Results/{ds_group}"):
    if ds_name not in ["ayeastCC", "west", "singh", "CSS", "PBS", "BP", "PDX"]: # Last 4 crash because not enough RAM, first 3 are too large to read as pandas dataframe! :(
      part_b_results = pd.read_csv(f"Results/{ds_group}/{ds_name}/{ds_name}_res.csv")
      part_d_results = pd.read_csv(f"Results_PartD/{ds_name}_res_aug.csv")
      for measure in MEASURE_TYPES:
        part_b_measure_results = part_b_results[part_b_results["Measure Type"] == measure]["Measure Value"]
        part_d_measure_results = part_d_results[part_d_results["Measure Type"] == measure]["Measure Value"]
        part_b_measure_mean = part_b_measure_results.mean()
        part_d_measure_mean = part_d_measure_results.mean()
        part_b_measure_max = part_b_measure_results.max()
        part_d_measure_max = part_d_measure_results.max()
        if (part_d_measure_max > part_b_measure_max):
          print(f"Maximum measurement improved!\nDataset: {ds_name}\nMeasurement: {measure}\nPart B max: {part_b_measure_max}\nPart D max: {part_d_measure_max}")
          maximum_improved.append((ds_name, measure, part_b_measure_max, part_d_measure_max))
        if (part_b_measure_mean != 0.0 and (fabs(part_b_measure_mean - part_d_measure_mean) / fabs(part_b_measure_mean)) > 0.075) or (part_b_measure_mean == 0 and fabs(part_d_measure_mean) > 0.05):
          print(f"Relative error of more than 7.5% detected!\nDataset: {ds_name}\nMeasurement: {measure}\nPart B mean: {part_b_measure_mean}\nPart D mean: {part_d_measure_mean}\n\n")
          if part_b_measure_mean > part_d_measure_mean:
            part_b_won.append((ds_name, measure, part_b_measure_mean, part_d_measure_mean))
          else:
            part_d_won.append((ds_name, measure, part_b_measure_mean, part_d_measure_mean))


In [ ]:
print("Part D won: \n" + '\n'.join([f"Dataset: {a}, Measure: {b}, Part B: {c}, Part D: {d}" for (a, b, c, d) in part_d_won]))

In [ ]:
print("Part B won: \n" + '\n'.join([f"Dataset: {a}, Measure: {b}, Part B: {c}, Part D: {d}" for (a, b, c, d) in part_b_won]))

In [ ]:
print(f"Part B win count: {len(part_b_won)}\nPart D win count: {len(part_d_won)}")

The code block below runs the experiment we've described in our conclusions for part D

In [ ]:
def run_part_d_analysis_experiemnt(X, y, ds_name, results_file_name):
  RESULTS_FILENAME = f"Results_PartD/{ds_name}_EXP.csv"
  if not exists(RESULTS_FILENAME):
    print(f"Reading file {results_file_name}...")
    res_df = pd.read_csv(results_file_name, header=0)
    best_auc_row_idx = res_df[(res_df["Measure Type"] == 'AUC') & (res_df["Number of features selected (K)"] >= 5)]["Measure Value"].idxmax()
    best_auc_row = res_df.iloc[best_auc_row_idx]
    winning_fs = best_auc_row["Filtering Algorithm"]
    winning_k = best_auc_row["Number of features selected (K)"]
    winning_clf = best_auc_row["Learning Algorithm"]
    fs_algo = FS[winning_fs](winning_k)
    X_new = fs_algo.fit_transform(X, y)
    X = pd.DataFrame(X_new, index=X.index)
    part_d(X, y, winning_fs, fs_algo, winning_clf, ds_name, winning_k, RESULTS_FILENAME)


for ds_group in listdir('Results'):
  for ds_name in listdir(f"Results/{ds_group}"):
    if ds_name not in ["ayeastCC", "west", "singh", "CSS", "PBS", "BP", "PDX"]: # Last 4 crash because not enough RAM, first 3 are too large to read as pandas dataframe! :(
      results_file_name = f"Results/{ds_group}/{ds_name}/{ds_name}_res.csv"
      X, y = get_x_y(ds_group, ds_name)
      # Run the same preprocessing as we run in Part B
      curr_preprocessing = make_pipeline(SimpleImputer(), VarianceThreshold(), PowerTransformer())
      REDUCE_FEATURES = X.shape[1] > 1000
      if REDUCE_FEATURES:
        curr_preprocessing.steps.append(['initial_feature_reducer', SelectKBest(k=1000)])
      X_new = curr_preprocessing.fit_transform(X, y)
      new_cols = X.columns[curr_preprocessing['initial_feature_reducer'].get_support()] if REDUCE_FEATURES else X.columns
      X = pd.DataFrame(X_new, columns=new_cols, index=X.index)

      run_part_d_analysis_experiemnt(X, y, ds_name, results_file_name)

In [ ]:
part_b_won = []
part_d_won = []
maximum_improved = []

for ds_group in listdir('Results'):
  for ds_name in listdir(f"Results/{ds_group}"):
    if ds_name not in ["ayeastCC", "west", "singh", "CS", "CSS", "PBS", "BP", "PDX"]: # Last 4 crash because not enough RAM, first 3 are too large to read as pandas dataframe! :(
      part_b_results = pd.read_csv(f"Results/{ds_group}/{ds_name}/{ds_name}_res.csv")
      part_d_results = pd.read_csv(f"Results_PartD/{ds_name}_EXP.csv")
      for measure in MEASURE_TYPES:
        part_b_measure_results = part_b_results[part_b_results["Measure Type"] == measure]["Measure Value"]
        part_d_measure_results = part_d_results[part_d_results["Measure Type"] == measure]["Measure Value"]
        part_b_measure_mean = part_b_measure_results.mean()
        part_d_measure_mean = part_d_measure_results.mean()
        part_b_measure_max = part_b_measure_results.max()
        part_d_measure_max = part_d_measure_results.max()
        if (part_d_measure_max > part_b_measure_max):
          print(f"Maximum measurement improved!\nDataset: {ds_name}\nMeasurement: {measure}\nPart B max: {part_b_measure_max}\nPart D max: {part_d_measure_max}")
          maximum_improved.append((ds_name, measure, part_b_measure_max, part_d_measure_max))
        if (part_b_measure_mean != 0.0 and (fabs(part_b_measure_mean - part_d_measure_mean) / fabs(part_b_measure_mean)) > 0.075) or (part_b_measure_mean == 0 and fabs(part_d_measure_mean) > 0.05):
          print(f"Relative error of more than 7.5% detected!\nDataset: {ds_name}\nMeasurement: {measure}\nPart B mean: {part_b_measure_mean}\nPart D mean: {part_d_measure_mean}\n\n")
          if part_b_measure_mean > part_d_measure_mean:
            part_b_won.append((ds_name, measure, part_b_measure_mean, part_d_measure_mean))
          else:
            part_d_won.append((ds_name, measure, part_b_measure_mean, part_d_measure_mean))


In [ ]:
print("Part D won: \n" + '\n'.join([f"Dataset: {a}, Measure: {b}, Part B: {c}, Part D: {d}" for (a, b, c, d) in part_d_won]))

In [ ]:
print("Part B won: \n" + '\n'.join([f"Dataset: {a}, Measure: {b}, Part B: {c}, Part D: {d}" for (a, b, c, d) in part_b_won]))

In [ ]:
print("Maximum improved: \n" + '\n'.join([f"Dataset: {a}, Measure: {b}, Part B: {c}, Part D: {d}" for (a, b, c, d) in maximum_improved]))

In [ ]:
print(f"Part B win count: {len(part_b_won)}\nPart D win count: {len(part_d_won)}")

# Part E

In [ ]:
original_algorithm_auc = None
suggested_improvement_auc = None

for ds_group in listdir('Results'):
  for ds_name in listdir(f"Results/{ds_group}"):
    results_file_name = f"Results/{ds_group}/{ds_name}/{ds_name}_res.csv"
    res_df = pd.read_csv(results_file_name, header=0)
    if original_algorithm_auc is not None:
      original_algorithm_auc = np.concatenate((original_algorithm_auc, res_df[(res_df["Measure Type"] == 'AUC') & (res_df["Filtering Algorithm"] == "CBFS")]["Measure Value"].values))
    else:
      original_algorithm_auc = res_df[(res_df["Measure Type"] == 'AUC') & (res_df["Filtering Algorithm"] == "CBFS")]["Measure Value"].values
    if suggested_improvement_auc is not None:
      suggested_improvement_auc = np.concatenate((suggested_improvement_auc, res_df[(res_df["Measure Type"] == 'AUC') & (res_df["Filtering Algorithm"] == "New_CBFS")]["Measure Value"].values))
    else:
      suggested_improvement_auc = res_df[(res_df["Measure Type"] == 'AUC') & (res_df["Filtering Algorithm"] == "New_CBFS")]["Measure Value"].values
  
bound = min(len(original_algorithm_auc), len(suggested_improvement_auc))
original_algorithm_auc = original_algorithm_auc[:bound]
suggested_improvement_auc = suggested_improvement_auc[:bound]
friedmanchisquare(original_algorithm_auc, suggested_improvement_auc, np.copy(original_algorithm_auc), np.copy(suggested_improvement_auc))

In [ ]:
posthoc_data = np.array([original_algorithm_auc, suggested_improvement_auc, np.copy(original_algorithm_auc), np.copy(suggested_improvement_auc)])
posthoc(posthoc_data.T)